In [77]:
# -*- coding: utf-8 -*-

import sqlite3
import numpy as np
import pandas as pd


In [78]:
def elim_doublons(data, attrib = 'Pays'):
    data = data.drop_duplicates()
    data = data[data[attrib] != 'Chine - RAS de Hong-Kong'] 
    data = data[data[attrib] != 'Chine, continentale']
    data = data[data[attrib] != 'Chine, Taiwan Province de']
    data = data[data[attrib] != 'Chine - RAS de Macao']
    return data


In [79]:
# exporter un fichier csv
# df.to_csv("export.csv", index = False)
# df.insert(0, 'id', range(1, 1+ len((df)))) 

## Préparation table population 

In [80]:
df_table_population = pd.read_csv('table_population.csv', header = 0)

In [81]:
df_table_population_unique = elim_doublons(df_table_population)

In [82]:
df_table_population_ready = df_table_population_unique[['Code Pays', 'Pays', 'Année', 'Valeur']]

In [83]:
e = df_table_population_ready[['Code Pays', 'Année']].drop_duplicates().dropna()
f = df_table_population_ready[['Code Pays', 'Année']]
if e.shape == f.shape:
    print('Ok clé primaire')
else:
    print('faux')

Ok clé primaire


In [84]:
df_table_population_ready.to_csv('table_population_ready.csv')

## Préparation table dispo_alim

In [85]:
df_table_dispo_alim = pd.read_csv('table_dispo_alim.csv', header = 0)

In [86]:
df_table_dispo_alim_unique = elim_doublons(df_table_dispo_alim)

In [87]:
df_table_dispo_alim = df_table_dispo_alim_unique[['Code Pays', 'Pays', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur']]

In [88]:
a = df_table_dispo_alim.pivot(columns = 'Élément', values = 'Valeur')

In [89]:
df_table_dispo_alim = df_table_dispo_alim.join(a)

In [90]:
del df_table_dispo_alim["Élément"]

In [91]:
del df_table_dispo_alim["Valeur"]


In [92]:
df4 = pd.read_csv("vegetaux.csv", header = 0) #Lecture du fichier csv avec prise en compte de l'en-tête comme index
df4 = df4.groupby(['Code Produit', 'Produit']).sum().reset_index()
df4 = df4[['Code Produit','Produit']]
df4['Nature Produit 2'] = 'végétal'
df_table_dispo_alim = df_table_dispo_alim.merge(df4, how='outer')
df_table_dispo_alim['Nature Produit 2'].replace([np.nan], 'animal', inplace=True)


In [93]:
#df_table_dispo_alim['Nature Produit'] = ((df_table_dispo_alim['Code Produit'] == 2511) | (df_table_dispo_alim['Code Produit'] == 2805) | (df_table_dispo_alim['Code Produit'] == 2513) | (df_table_dispo_alim['Code Produit'] == 2514) | (df_table_dispo_alim['Code Produit'] == 2515) | (df_table_dispo_alim['Code Produit'] == 2516) | (df_table_dispo_alim['Code Produit'] == 2517) | (df_table_dispo_alim['Code Produit'] == 2518) | (df_table_dispo_alim['Code Produit'] == 2520))

In [94]:
#df_table_dispo_alim = df_table_dispo_alim.replace({'Nature Produit': {True: 'végétal'}})
#df_table_dispo_alim = df_table_dispo_alim.replace({'Nature Produit': {False: 'animal'}})

In [95]:
del df_table_dispo_alim["Produit"]
df_table_dispo_alim = df_table_dispo_alim.merge(df_table_population_ready, how='outer')

In [96]:
df_table_dispo_alim['Disponibilité alimentaire en tonnes'] = df_table_dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)'] * df_table_dispo_alim['Valeur']

In [97]:
a = df_table_dispo_alim.groupby(['Code Pays', 'Pays', 'Année', 'Nature Produit 2', 'Code Produit']).sum().reset_index()
a

,Code Pays,Pays,Année,Nature Produit 2,Code Produit,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Valeur,Disponibilité alimentaire en tonnes
0,1,Arménie,2010,animal,2731,103.0,18.01,7.97,7.31,11852,5.336363e+04
1,1,Arménie,2010,animal,2732,19.0,2.67,1.67,0.99,11852,7.911210e+03
2,1,Arménie,2010,animal,2733,42.0,6.07,3.92,1.46,11852,1.798541e+04
3,1,Arménie,2010,animal,2734,47.0,14.01,2.95,4.74,11852,4.151163e+04
4,1,Arménie,2010,animal,2735,0.0,0.00,0.00,0.00,11852,0.000000e+00
5,1,Arménie,2010,animal,2736,13.0,4.44,0.37,2.18,11852,1.315572e+04
6,1,Arménie,2010,animal,2737,69.0,3.36,7.55,0.22,11852,9.955680e+03
7,1,Arménie,2010,animal,2740,27.0,1.37,3.05,0.03,11852,4.059310e+03
8,1,Arménie,2010,animal,2743,0.0,0.01,0.00,0.00,11852,2.963000e+01
9,1,Arménie,2010,animal,2744,47.0,12.38,3.33,3.63,11852,3.668194e+04


In [98]:
del a['Disponibilité alimentaire en quantité (kg/personne/an)']
del a['Valeur']
a

,Code Pays,Pays,Année,Nature Produit 2,Code Produit,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité alimentaire en tonnes
0,1,Arménie,2010,animal,2731,103.0,7.97,7.31,5.336363e+04
1,1,Arménie,2010,animal,2732,19.0,1.67,0.99,7.911210e+03
2,1,Arménie,2010,animal,2733,42.0,3.92,1.46,1.798541e+04
3,1,Arménie,2010,animal,2734,47.0,2.95,4.74,4.151163e+04
4,1,Arménie,2010,animal,2735,0.0,0.00,0.00,0.000000e+00
5,1,Arménie,2010,animal,2736,13.0,0.37,2.18,1.315572e+04
6,1,Arménie,2010,animal,2737,69.0,7.55,0.22,9.955680e+03
7,1,Arménie,2010,animal,2740,27.0,3.05,0.03,4.059310e+03
8,1,Arménie,2010,animal,2743,0.0,0.00,0.00,2.963000e+01
9,1,Arménie,2010,animal,2744,47.0,3.33,3.63,3.668194e+04


In [99]:
e = a[['Code Pays', 'Année','Code Produit']].drop_duplicates().dropna()
f = a[['Code Pays', 'Année','Code Produit']]
if e.shape == f.shape:
    print('Ok clé primaire')
else:
    print('faux')

Ok clé primaire


In [100]:
a.to_csv('table_dispo_alim_ready.csv')

## Préparation table equilibre_prod

In [101]:
df_table_equilibre_prod = pd.read_csv('table_equilibre_prod_2.csv', header = 0)
df_table_equilibre_prod_unique = elim_doublons(df_table_equilibre_prod)
df_table_equilibre_prod =df_table_equilibre_prod_unique[['Code Pays', 'Pays', 'Année', 'Produit', 'Code Produit','Élément', 'Valeur']]

In [102]:
df_table_equilibre_prod

,Code Pays,Pays,Année,Produit,Code Produit,Élément,Valeur
0,2,Afghanistan,2010,Blé,2511,Disponibilité intérieure,5596
1,2,Afghanistan,2011,Blé,2511,Disponibilité intérieure,5547
2,2,Afghanistan,2012,Blé,2511,Disponibilité intérieure,5884
3,2,Afghanistan,2013,Blé,2511,Disponibilité intérieure,5992
4,2,Afghanistan,2010,Blé,2511,Semences,313
5,2,Afghanistan,2011,Blé,2511,Semences,328
6,2,Afghanistan,2012,Blé,2511,Semences,316
7,2,Afghanistan,2013,Blé,2511,Semences,322
8,2,Afghanistan,2010,Blé,2511,Pertes,678
9,2,Afghanistan,2011,Blé,2511,Pertes,508


In [103]:
a = df_table_equilibre_prod.pivot(columns = 'Élément', values = 'Valeur')

In [104]:
df_table_equilibre_prod = df_table_equilibre_prod.join(a)

In [105]:
del df_table_equilibre_prod["Élément"]
del df_table_equilibre_prod["Valeur"]

In [106]:
df_table_equilibre_prod = df_table_equilibre_prod.groupby(['Code Pays', 'Pays', 'Année', 'Produit', 'Code Produit']).sum()


In [107]:
df_table_equilibre_prod = df_table_equilibre_prod.reset_index()

In [108]:
df_table_equilibre_prod

,Code Pays,Pays,Année,Produit,Code Produit,Aliments pour animaux,Autres Utilisations,Disponibilité intérieure,Nourriture,Pertes,Semences,Traitement
0,1,Arménie,2010,Abats Comestible,2736,0.0,1.0,14.0,13.0,0.0,0.0,0.0
1,1,Arménie,2010,"Agrumes, Autres",2614,0.0,0.0,3.0,3.0,0.0,0.0,0.0
2,1,Arménie,2010,"Alcool, non Comestible",2659,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1,Arménie,2010,Aliments pour enfants,2680,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Arménie,2010,Ananas,2618,0.0,0.0,1.0,1.0,0.0,0.0,0.0
5,1,Arménie,2010,Animaux Aquatiques Autre,2769,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,Arménie,2010,Arachides Decortiquees,2556,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,1,Arménie,2010,Avoine,2516,2.0,0.0,3.0,1.0,0.0,0.0,0.0
8,1,Arménie,2010,Bananes,2615,0.0,0.0,8.0,8.0,0.0,0.0,0.0
9,1,Arménie,2010,Bananes plantains,2616,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
e = df_table_equilibre_prod[['Code Pays', 'Année','Code Produit']].drop_duplicates().dropna()
f = df_table_equilibre_prod[['Code Pays', 'Année','Code Produit']]
if e.shape == f.shape:
    print('Ok clé primaire')
else:
    print('faux')

Ok clé primaire


In [110]:
df_table_equilibre_prod.to_csv('table_equilibre_prod_ready.csv')

## Préparation table sous alimentation

In [111]:
df_table_sous_alim = pd.read_csv('table_sous_alim.csv', header = 0)
df_table_sous_alim_unique = elim_doublons(df_table_sous_alim, attrib = 'Zone')
df_table_sous_alim = df_table_sous_alim_unique[['Code zone', 'Zone', 'Année', 'Valeur']]

In [112]:
df_table_sous_alim

,Code zone,Zone,Année,Valeur
0,2,Afghanistan,2009-2011,6.2
1,2,Afghanistan,2010-2012,6.4
2,2,Afghanistan,2011-2013,6.8
3,2,Afghanistan,2012-2014,7.2
4,2,Afghanistan,2013-2015,7.5
5,2,Afghanistan,2014-2016,7.5
6,2,Afghanistan,2015-2017,NaN
7,202,Afrique du Sud,2009-2011,2.0
8,202,Afrique du Sud,2010-2012,1.9
9,202,Afrique du Sud,2011-2013,1.9


In [113]:
e = df_table_sous_alim[['Code zone', 'Année']].drop_duplicates().dropna()
f = df_table_sous_alim[['Code zone', 'Année']]
if e.shape == f.shape:
    print('Ok clé primaire')
else:
    print('faux')

Ok clé primaire


In [114]:
df_table_sous_alim.to_csv('table_sous_alim_ready.csv')